## Data preparation
We need to provide coordinates and embedding as the training data and forces as the label. You can git clone https://github.com/Hori-Lab/UNISIS.git and go to ./utils/ML/ where you will see multiple python files to process data.

### Force data
The first thing is to run **DCD-force mode** (Naoto can help you compile the environment) on the dcd file to generate 6 forces including angle, bond, base pair, electrostatics and excluded volumn. Then, the force files need to be trandformed into npy files which has smaller size and also easier to be processed in the neural network. Please use **force_out_to_npy.py** https://github.com/Hori-Lab/UNISIS/blob/main/utils/ML/force_out_to_npy.py to transform.

Once you obtain the npy files, angle and base pair and electrostatics forces will be added up to get delta forces as the label.

In [ ]:
import numpy as np
import os

In [ ]:
array1 = np.load('./force_angl.npy')
array2 = np.load('./force_bond.npy')
array3 = np.load('./force_bp.npy')
array4 = np.load('./force_dihe.npy')
array5 = np.load('./force_ele.npy')
array6 = np.load('./force_exv.npy')

In [ ]:
delta_forces = array1 + array3 + array5

In [ ]:
np.save('./force_deltaforces.npy', delta_forces) 

### coordinates data

Please use **dcd_to_npy.py** https://github.com/Hori-Lab/UNISIS/blob/main/utils/ML/dcd_to_npy.py to convert dcd file to npy file which contains the coordinates for every frames.

### Embedding

Please use **fasta_to_embedding.py** https://github.com/Hori-Lab/UNISIS/blob/main/utils/ML/fasta_to_embedding.py to convert fasta file to npy file which uses embedding numbers to represent nucleotides.

## After Training and Simulation

## Training Curve
In your log folder, please check the metrics.csv which has the information about training loss, validation loss and test loss etc. You can plot them using the following cell.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('./metrics.csv')

# Create a copy of the dataframe and remove the last value of 'test_total_l1_loss'
df_modified = df.copy()
df_modified.loc[df_modified.index[-1], 'test_total_l1_loss'] = None  # Set the last value to None

# Create subplots
fig, ax = plt.subplots(nrows=2, figsize=[6, 4])

# Plot training and validation MSE loss
df.plot(x='epoch', y=['train_total_mse_loss', 'val_total_mse_loss'], ax=ax[0])

# Plot modified test L1 loss (excluding the last value)
df_modified.dropna(subset=['test_total_l1_loss']).plot(x='epoch', y='test_total_l1_loss', ax=ax[0], color='green', label='test_total_l1_loss')

# Set label for the y-axis of the first subplot
ax[0].scatter([299], [df['val_total_mse_loss'].loc[299]], s=200, c='r', marker=(5, 1), label='selected epoch')
ax[0].set_ylabel('Loss')

# Plot learning rate
df.plot(x='epoch', y='lr', ax=ax[1])

# Set label for the y-axis of the second subplot
ax[1].set_ylabel('Learning Rate')

# Display the plots
plt.show()

### RMSD

RMSD measures the average distance between atoms of two molecular structures after optimal superposition. It quantifies structural similarity. Firstly, you need to git clone https://github.com/naotohori/life-of-py.git and use **bestfit_dcd_by_pdb_part_onlyRMSD.py**

Here is a python file to process many dcd files to output RMSD valuse. Please replace the neccessary paths. https://github.com/Takashibobbob/UNISIS-ML-analysis/blob/main/RMSD/all_sim_npy_to_dcd_rmsd_native.py

After you obtain the RMSD results, use the following cell to plot.

In [ ]:
def plot_rmsd_native_mirror(dirs):
    #nreplica = 32
    ncols = 5
    nrows = 2
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=[3*ncols, 2*nrows])
    for irep, dir in enumerate(dirs):
        irow = irep // ncols
        icol = irep % ncols
        rmsd1 = []
        time1 = []
        with open(f"{dir}/md.rmsd",'r') as file:
            for l, j in enumerate(file):
                time1.append(float(l) * 10 ** (-5))
                rmsd1.append(float(j))
            axs[irow,icol].plot(time1, rmsd1, color = 'tomato', label = 'native')

        rmsd2 = []
        time2 = []
        with open(f"{dir}/md_mirror.rmsd",'r') as file:
            for l, j in enumerate(file):
                time2.append(float(l) * 10 ** (-5))
                rmsd2.append(float(j))
            axs[irow,icol].plot(time2, rmsd2, color = 'royalblue', label = 'mirror')
            
            axs[irow,icol].set_ylim(0,40)
            axs[irow,icol].text(0, 5, f'{irep}')
        if irep == 0:
            axs[irep,icol].legend()
             
            # Add labels for x-axis and y-axis for the entire figure
    for ax in axs[-1]:  # Set x-labels only on the last row
        ax.set_xlabel("Time (μs)")
    for ax in axs[:, 0]:  # Set y-labels only on the first column
        ax.set_ylabel("RMSD (Å)")

    plt.tight_layout()
    plt.show()
        
dirs = sorted(glob.glob('./Simulation/Simulation_3_30_36_23456/run_all/run0**'))

plot_rmsd_native_mirror(dirs)

### Q-score

Q-score evaluates how similar two structures are based on their contact patterns (e.g., between residues or nucleotides), instead of distances alone. Please use the python file https://github.com/Takashibobbob/UNISIS-ML-analysis/blob/main/Q_score/dcd_to_Qscore.py to process multiple dcd files. Note that /users/ssyjb1/sis/sis this path is going to be replaced by the path where you git clone https://github.com/Hori-Lab/UNISIS.git and ask Naoto to help you compile.

Note that sisbp_nbp.py should be in https://github.com/naotohori/life-of-py.git that you have git cloned already. Same here, please replace the necessary paths.

After you obtain the Q-score values, please use the following cell to plot out.

In [ ]:
def plot_Q_score(dirs):
    nplots = len(dirs)
    if nplots == 0:
        print("No directories found.")
        return
    
    ncols = min(nplots, 5)  # Max 5 columns
    nrows = (nplots + ncols - 1) // ncols  # Auto-adjust rows
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=[3*ncols, 2*nrows])
    axs = axs.flatten()  # Flatten to handle indexing properly
    
    for irep, dir in enumerate(dirs):
        Q_score = []
        time1 = []
        md_nbp_path = os.path.join(dir, 'md.nbp')

        if not os.path.exists(md_nbp_path):
            print(f"Skipping {md_nbp_path} (file not found)")
            continue

        with open(md_nbp_path, 'r') as file:
            for l, j in enumerate(file):
                time1.append(float(l) * 10 ** (-5))
                Q_score.append(float(j.split()[1]))
            axs[irep].plot(time1, Q_score, color='#F9AE7B', label='native')
        
        axs[irep].set_ylim(bottom=0.0, top=max(Q_score) * 1.1)
        axs[irep].text(0, max(Q_score) * 0.9, f'{irep}')
       
    last_row_start = (nrows - 1) * ncols
    for i in range(last_row_start, nplots):
        axs[i].set_xlabel("Time (μs)")
    
    for i in range(0, nplots, ncols):
        axs[i].set_ylabel("Q-score")
        
    plt.tight_layout()
    plt.show()

# Get directories
dirs = sorted(glob.glob('/users/ssyjb1/RNA-ML-VPK-HP2/Simulation/Simulation_3_30_36_Q/run_all/run*'))

# Run function
plot_Q_score(dirs)

### TICA

> I would suggest that you can create a new environment for TICA rather than the one you use for Training/Simulation and make sure to install all the require libraries in the following python files.

TICA is a dimensionality reduction technique that extracts the slowest dynamical modes from molecular simulation data. In this project, we use TICA to produce energy landscapes for both one dimension and two dimension.

Firstly, it is important to select your desired trajectory (e.g ML simulation) which need to be superimposed to reference structure (native folded structure) to get a superimposed version dcd file. Please use https://github.com/naotohori/life-of-py/blob/master/bestfit_dcd_by_pdb.py to do this step. The reason is to avoid the noise when doing reduction of dimensions

Then, convert the superimposed dcd file to npy file which has the coordinates that is similar to what you have done in data preparation. Please use https://github.com/Hori-Lab/UNISIS/blob/main/utils/ML/dcd_to_npy.py.


In order to obtain the two TICA vectors, please use the python file https://github.com/Takashibobbob/UNISIS-ML-analysis/blob/main/TICA/TICA_cov_sis.py. The first and second TICA vectors will be stored in e.g. ML_TICA.txt. Note that it will take very long time especially for long simulation. please use **condor system to submit**.

After that, it is able to use the TICA vetors to project on another simulation (e.g. reference simulation). Please use the python file https://github.com/Takashibobbob/UNISIS-ML-analysis/blob/main/TICA/TICA_ML.py to obtain the necessary files to plot energy landscape. Note that you need to extract the mean value of your desired trajectory (e.g. ML simulation) coordinates corresponding to the input **mean_ref**, use the following cell. Also, you need to superimpose the dcd file to reference structure to obtain coordinates. **TICA_cov** is the ML_TICA.txt

In [ ]:
import numpy as np
coordinates_align = np.load('/users/ssyjb1/RNA-ML-VPK-HP2/Simulation/active_force/Simulation_3_30_36_95472/run_all/coordinates.npy')
mean_value = np.mean(coordinates_align,axis=0)
mean_free = coordinates_align - mean_value
np.savetxt('/users/ssyjb1/RNA-ML-VPK-HP2/Simulation/active_force/Simulation_3_30_36_95472/run_all/mean_value.txt', mean_value)

Once you got the output files from **TICA_cov_sis.py** and **TICA_ML.py**, please use https://github.com/Takashibobbob/UNISIS-ML-analysis/blob/main/TICA/process_tica.py to plot out the energy landscape. Remember to do RMSD analysis on the dcd file to get RMSD values which is an input.